# PERP-UNI NVP Spread Investment Monitor

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import data
import analytics

%load_ext autoreload
%autoreload 2

## Inputs

In [2]:
start_date = '2020-09-09'
end_date = '2021-09-25'
dexs = [
    'uniswap_v1',
    'uniswap_v2',
    'uniswap',
    'perpetual_protocol',
]
coins = [
    'uniswap',
    'perpetual-protocol',
]
benchmark = 'defipulse-index'
nvf_avg_window = 30
annualization_factor = 365
uni_trade_fee = 0.003
perp_trade_fee = 0.001
initial_capital = 1978.28
risk_free_rate = 0

## Get Data

In [3]:
start_date = pd.to_datetime(start_date).date()
end_date = pd.to_datetime(end_date).date()

# Call CoinGecko for market data
coins.append(benchmark)
dex_trading_volume = data.dex_trading_volume(dexs, start_date, end_date)
coin_market_data = data.coin_market_data(coins, start_date, end_date)
dex_trading_volume = dex_trading_volume[start_date:end_date]
coin_market_data = coin_market_data[start_date:end_date]

# Prepare DEX trading volume data
dex_trading_volume['uniswap'].fillna(0, inplace=True)
dex_trading_volume['uniswap_v2'].fillna(0, inplace=True)
dex_trading_volume['uniswap_v1'].fillna(0, inplace=True)
uni_tv = dex_trading_volume['uniswap'] + dex_trading_volume['uniswap_v2'] + dex_trading_volume['uniswap_v1']
uni_tv = uni_tv[uni_tv != 0]
perp_tv = dex_trading_volume['perpetual_protocol'].dropna()

# Prepare coin and benchmark market data
uni_price = coin_market_data['uniswap price'].dropna()
uni_mc = coin_market_data['uniswap market cap'].dropna()
perp_price = coin_market_data['perpetual-protocol price'].dropna()
perp_mc = coin_market_data['perpetual-protocol market cap'].dropna()
benchmark_price = coin_market_data['defipulse-index price'].dropna()
benchmark_mc = coin_market_data['defipulse-index market cap'].dropna()

# Import and format trade data
funding_payments = pd.read_csv('input/funding_payments.csv', index_col='time', parse_dates=True)
pnl = pd.read_csv('input/pnl.csv', index_col='day')
pnl = pnl.drop(' Total')
pnl.index = pnl.index.map(lambda x: pd.to_datetime(float(x), unit='s'))

# Prepare trade data
funding_payments = funding_payments.groupby('future').resample('D').sum()
perp_fp = funding_payments.loc['PERP-PERP']['payment']
perp_fp.index = perp_fp.index.tz_localize(None)
perp_fp = perp_fp[start_date:end_date]
uni_fp = funding_payments.loc['UNI-PERP']['payment']
uni_fp.index = uni_fp.index.tz_localize(None)
uni_fp = uni_fp[start_date:end_date]
perp_pnl = pnl['PERP-PERP'].sort_index()[start_date:end_date]
uni_pnl = pnl['UNI-PERP'].sort_index()[start_date:end_date]
perp_net_pnl = perp_pnl - perp_fp
uni_net_pnl = uni_pnl - uni_fp

Getting DEX trading volume...
Getting coin market data...
- bitcoin
Got coin market data
- uniswap_v1
- uniswap
- uniswap_v2
- perpetual_protocol
Got DEX trading volume
Getting coin market data...
- uniswap
- defipulse-index
- perpetual-protocol
Got coin market data


## Compute Analytics

In [4]:
# NVF
uni_nvf = analytics.nvf(uni_mc, uni_tv, uni_trade_fee, nvf_avg_window, annualization_factor)
perp_nvf = analytics.nvf(perp_mc, perp_tv, perp_trade_fee, nvf_avg_window, annualization_factor)

# NVF spread (absolute & % diff)
abs_nvf_spread = (perp_nvf - uni_nvf).dropna()
perc_nvf_spread = (abs_nvf_spread / uni_nvf).dropna()

# Determine number of trading days
first_trade_date = min(uni_pnl.index)
today = pd.Timestamp.today()
trade_days = (today - first_trade_date).days

# Overview portfolio stats
portfolio_perf = analytics.pnl_performance(perp_net_pnl + uni_net_pnl, initial_capital)
portfolio_total_ret = analytics.total_return(portfolio_perf)
portfolio_ann_ret = (annualization_factor / trade_days) * portfolio_total_ret
portfolio_log_ret = analytics.returns(portfolio_perf, 'log')
portfolio_arith_ret = analytics.returns(portfolio_perf, 'arith')
benchmark_perf = benchmark_price[first_trade_date:end_date]
benchmark_log_ret = analytics.returns(benchmark_perf, 'log')
benchmark_arith_ret = analytics.returns(benchmark_perf, 'arith')
portfolio_ann_vol = np.sqrt(annualization_factor) * analytics.volatility(portfolio_log_ret)
portfolio_sr = analytics.sharpe_ratio(portfolio_log_ret, risk_free_rate, annualization_factor)
portfolio_ir = analytics.information_ratio(portfolio_arith_ret, benchmark_arith_ret)
portfolio_beta = analytics.beta(portfolio_log_ret, benchmark_log_ret)
portfolio_alpha = analytics.alpha(portfolio_log_ret, benchmark_log_ret, risk_free_rate, annualization_factor)
portfolio_tr = analytics.treynor_ratio(portfolio_log_ret, benchmark_log_ret, risk_free_rate, annualization_factor)
portfolio_md = analytics.max_drawdown(portfolio_arith_ret)

# Overview benchmark stats
benchmark_total_ret = analytics.total_return(benchmark_perf)
benchmark_ann_ret = (annualization_factor / trade_days) * benchmark_total_ret
benchmark_ann_vol = np.sqrt(annualization_factor) * analytics.volatility(benchmark_log_ret)
benchmark_sr = analytics.sharpe_ratio(benchmark_log_ret, risk_free_rate, annualization_factor)
benchmark_md = analytics.max_drawdown(benchmark_arith_ret)

## Present Analytics

In [5]:
# Portfolio and benchmark overview stats
portfolio_stats = {
    'Total return': portfolio_total_ret,
    'Annualized return': portfolio_ann_ret,
    'Volatility': portfolio_ann_vol,
    'Max drawdown': portfolio_md,
    'Sharpe ratio': portfolio_sr,
    'Information ratio': portfolio_ir,
    'Treynor ratio': portfolio_tr,
    'Alpha': portfolio_alpha,
    'Beta': portfolio_beta,
}
benchmark_stats = {
    'Total return': benchmark_total_ret,
    'Annualized return': benchmark_ann_ret,
    'Volatility': benchmark_ann_vol,
    'Max drawdown': benchmark_md,
    'Sharpe ratio': benchmark_sr,
    'Information ratio': '',
    'Treynor ratio': '',
    'Alpha': '',
    'Beta': '',
}
trade_stats = pd.DataFrame([portfolio_stats, benchmark_stats], index=['Portfolio', 'Benchmark'])
trade_stats

# TODO: Plot stuff

,Total return,Annualized return,Volatility,Max drawdown,Sharpe ratio,Information ratio,Treynor ratio,Alpha,Beta
Portfolio,0.003358,0.153220,0.342384,-0.043828,0.446760,0.452953,1.55893,1.39522,0.0981208
Benchmark,-0.242317,-11.055729,1.470000,-0.232224,-8.612594,,,,


## Output Data & Analytics